In [1]:
# Capital Spending Project: Joining on CPDB
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import re
import datetime
import matplotlib.font_manager
import time

In [2]:
# --- checkbook nyc data
file_names = ['../data/checkbooknyc/Citywide-Agencies/spending_29351_05312023_040004_part_0.csv', 
'../data/checkbooknyc/Citywide-Agencies/spending_29351_05312023_040004_part_1.csv',
'../data/checkbooknyc/Citywide-Agencies/spending_29351_05312023_040004_part_2.csv',
'../data/checkbooknyc/Citywide-Agencies/spending_29351_05312023_040004_part_3.csv']
df0 = pd.read_csv(file_names[0])
df1 = pd.read_csv(file_names[1])
df2 = pd.read_csv(file_names[2])
df3 = pd.read_csv(file_names[3])
frames = [df0, df1, df2, df3]
df = pd.concat(frames)

In [3]:
df.head()

,Agency,Associated Prime Vendor,Budget Code,Capital Project,Check Amount,Contract ID,Contract Purpose,Department,Document ID,Emerging Business,Expense Category,Fiscal year,Industry,Issue Date,M/WBE Category,Payee Name,Spending Category,Sub Contract Reference ID,Sub Vendor,Woman Owned Business
0,Transit Authority,NaN,"BUSB (MTA, NYCT: PURCHASE 126 HYBRID BUSES)",998CAP2024 005,99999999.95,CT199820231408687,Bus Purch Track Replace Switch Replace Rail Su...,400-998-169,20230231715-1-DSB-EFT,No,CAPITAL PURCHASED EQUIPMENT,2023,Standardized Services,2022-12-21,Individuals and Others,METROPOLITAN TRANSPORTATION AUTHORITY,Capital Contracts,NaN,No,No
1,Transit Authority,NaN,"SAPS (MTA / NYCT: SUBWAY ACTION PLAN, SIGNAL A)",998CAPSAP 003,99999999.59,CT199820191409664,Subway Action Plan Signal Improvement & Modern...,400-998-169,20190025861-1-015-MD1,No,IOTB CONSTRUCTION,2019,Standardized Services,2018-10-30,Individuals and Others,METROPOLITAN TRANSPORTATION AUTHORITY,Capital Contracts,NaN,No,No
2,Department of Education,NaN,E706 (IMPLEMENTATION OF THE SEVENTH FIVE-YEAR),040SCA22 030,99999999.00,PON1040SCA20-24MC,NaN,IMPLEMENTATION OF THE SEVENTH FIVE-YEAR,20220448629-1-DSB-EFT,No,ADMINISTRATIVE EXPENSES,2022,NaN,2022-05-27,Individuals and Others,NYC SCHOOL CONSTRUCTION AUTHORITY,Capital Contracts,NaN,No,No
3,Department of Education,NaN,E706 (IMPLEMENTATION OF THE SEVENTH FIVE-YEAR),040SCA21 030,99999999.00,PON1040SCA20-24MC,NaN,IMPLEMENTATION OF THE SEVENTH FIVE-YEAR,20220448626-1-DSB-EFT,No,CONSTRUCTION-BUILDINGS,2022,NaN,2022-05-27,Individuals and Others,NYC SCHOOL CONSTRUCTION AUTHORITY,Capital Contracts,NaN,No,No
4,Department of Education,NaN,E706 (IMPLEMENTATION OF THE SEVENTH FIVE-YEAR),040SCA21 030,99999999.00,PON1040SCA20-24MC,NaN,IMPLEMENTATION OF THE SEVENTH FIVE-YEAR,20220354644-1-DSB-EFT,No,CONSTRUCTION-BUILDINGS,2022,NaN,2022-03-25,Individuals and Others,NYC SCHOOL CONSTRUCTION AUTHORITY,Capital Contracts,NaN,No,No


In [4]:
# limited data cleaning 
df = df[df['Check Amount']<99000000] # exclude checks over 99 mil
df = df[df['Check Amount']>=0] # exclude negative checks, include 0 checks
df['FMS ID'] = df['Capital Project'].str.replace(r'\s*\d+$','') # new column for joining on CPDB

/var/folders/7p/9414x_x93rvb4p6kc2ny_wl00000gn/T/ipykernel_14181/851556831.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['FMS ID'] = df['Capital Project'].str.replace(r'\s*\d+$','') # new column for joining on CPDB


In [5]:
df.head()

,Agency,Associated Prime Vendor,Budget Code,Capital Project,Check Amount,Contract ID,Contract Purpose,Department,Document ID,Emerging Business,...,Fiscal year,Industry,Issue Date,M/WBE Category,Payee Name,Spending Category,Sub Contract Reference ID,Sub Vendor,Woman Owned Business,FMS ID
176,Department of Education,NaN,E704 (IMPLEMENTATATION OF THE FIFTH FIVE-YEAR),040SCA13 010,98999000.0,PON1040SCA10-14MC,NaN,IMPLEMENTATION OF THE FIFTH FIVE-YEAR ED,20140165270-1-DSB-EFT,No,...,2014,NaN,2013-11-04,Individuals and Others,NYC SCHOOL CONSTRUCTION AUTHORITY,Capital Contracts,NaN,No,No,040SCA13
177,Department of Education,NaN,E704 (IMPLEMENTATATION OF THE FIFTH FIVE-YEAR),040SCA10 010,98261555.0,PON1040SCA10-14MC,NaN,IMPLEMENTATION OF THE FIFTH FIVE-YEAR ED,20120044911-1-DSB-AD,No,...,2012,NaN,2011-08-04,Individuals and Others,SCHOOL CONSTRUCTION AUTHORITY,Capital Contracts,NaN,No,No,040SCA10
178,SCHOOL CONSTRUCTION AUTHORITY,NaN,E704 (402044E704),040WRAP2010 020,98175810.0,PON104420141435313,NaN,IMPLEMENTATATION OF THE FIFTH FIVE-YEAR,20140149753-1-DSB-AD,No,...,2014,NaN,2014-02-05,Non-M/WBE,ACE AMERICAN INSURANCE CO.,Capital Contracts,NaN,No,No,040WRAP2010
179,SCHOOL CONSTRUCTION AUTHORITY,NaN,E705 (6TH PLAN - BL 2364),040WRAP2015 020,98000000.0,PON104420171733408,NaN,IMPLEMENTATION OF THE SIXTH FIVE-YEAR ED,20170126575-1-DSB-AD,No,...,2017,NaN,2017-01-30,Non-M/WBE,ACE AMERICAN INSURANCE CO.,Capital Contracts,NaN,No,No,040WRAP2015
180,Department of Education,NaN,E704 (IMPLEMENTATATION OF THE FIFTH FIVE-YEAR),040SCA12 010,98000000.0,PON1040SCA10-14MC,NaN,IMPLEMENTATION OF THE FIFTH FIVE-YEAR ED,20130085631-1-DSB-AD,No,...,2013,NaN,2012-11-13,Individuals and Others,SCHOOL CONSTRUCTION AUTHORITY,Capital Contracts,NaN,No,No,040SCA12


In [6]:
cols_dict = {}
for col in df.columns:
    if col == 'Check Amount':
        cols_dict[col] = 'sum'
    else:
        cols_dict[col] = 'first' # keep first value for all other cols

In [7]:
# groupby
cols_for_grouping = ['FMS ID', 'Budget Code', 'Agency', 'Fiscal year', 'Expense Category', 'Spending Category']
projects_df = df.groupby(cols_for_grouping, as_index=False)
checkbook_nyc_groupby_fms_id = projects_df.agg(cols_dict)
other_cols = [x for x in df.columns if x not in cols_for_grouping and x != 'Check Amount']
checkbook_nyc_groupby_fms_id.drop(other_cols, axis=1, inplace=True)

In [8]:
print(checkbook_nyc_groupby_fms_id.shape)
checkbook_nyc_groupby_fms_id.head(5)

(80932, 7)


,Agency,Budget Code,Check Amount,Expense Category,Fiscal year,Spending Category,FMS ID
0,Department of Citywide Administrative Services,DUMY (INITIAL COI AUTHORIZATION; FY 2004C),3000.00,OTPS HOLDING CODE,2011,Capital Contracts,002FINC01
1,Department of Citywide Administrative Services,DUMY (INITIAL COI AUTHORIZATION; FY 2004C),4158840.60,OTPS HOLDING CODE,2012,Capital Contracts,002FINC01
2,Department of Citywide Administrative Services,DUMY (INITIAL COI AUTHORIZATION; FY 2004C),639170.54,TRUST & AGENCY DETAIL FUND ADJ,2012,Capital Contracts,002FINC01
3,Department of Citywide Administrative Services,DUMY (INITIAL COI AUTHORIZATION; FY 2004C),3071699.41,OTPS HOLDING CODE,2013,Capital Contracts,002FINC01
4,Department of Citywide Administrative Services,DUMY (INITIAL COI AUTHORIZATION; FY 2004C),833857.31,TRUST & AGENCY DETAIL FUND ADJ,2013,Capital Contracts,002FINC01


In [12]:
# check that each group has consistent values
check_distinct_values_after_grouping = projects_df.agg({col: 'nunique' for col in df.columns})

mean_result = check_distinct_values_after_grouping.mean()
distinct = []
nondistinct = []
for col in cols_for_grouping:
    if (mean_result[col] == 1 or mean_result[col] == 0):
        nondistinct.append((col))
    else:
        distinct.append((col,mean_result[col]))

print("Cols with nondistinct values across groups:")
print(nondistinct)
print("Cols with distinct values across groups and their mean:")
print(distinct)

Cols with nondistinct values across groups:
['FMS ID', 'Budget Code', 'Agency', 'Fiscal year', 'Expense Category', 'Spending Category']
Cols with distinct values across groups and their mean:
[]


In [10]:
agg_df = projects_df.agg(cols_dict)
agg_df.drop('Issue Date', axis=1, inplace=True)

# agg_df.to_csv('checkbook_nyc_groupby_capital_project_fy.csv')


In [11]:
agg_df.shape

(80932, 20)